<a href="https://colab.research.google.com/github/lwyuki0524/extractive-summarization/blob/main/%E8%B3%87%E6%96%99%E7%A7%91%E5%AD%B8%E6%9C%9F%E6%9C%AB%E5%A0%B1%E5%91%8A_%E5%9F%BA%E6%96%BC%E8%81%9A%E9%A1%9E%E5%92%8C%E9%97%9C%E9%8D%B5%E8%A9%9E%E6%8F%90%E5%8F%96%E9%80%B2%E8%A1%8C%E6%8A%BD%E5%8F%96%E5%BC%8F%E6%91%98%E8%A6%81%E5%84%AA%E5%8C%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 載入套件

In [ ]:
!pip install -U sentence-transformers
!pip install transformers sentencepiece rouge -qq

In [ ]:
import numpy as np
import pandas as pd
import os
import nltk
import nltk.data
from nltk.tokenize import word_tokenize
nltk.download('punkt')

import copy
from rouge import Rouge
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

from sklearn.utils import shuffle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## 載入資料集

### CNN-DailyMail

In [ ]:
!unzip train.zip

Archive:  train.zip
  inflating: train.csv               


In [ ]:
cnn_dailymail_train_df = pd.read_csv('train.csv')
cnn_dailymail_train_df.head()

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [ ]:
# 打亂順序，選擇前500筆資料
shuffle_cnn_dailymail_train_df = shuffle(cnn_dailymail_train_df)
new_cnn_dailymail_train_df = shuffle_cnn_dailymail_train_df.reset_index(drop=False)[:500]
# 轉CSV檔
new_cnn_dailymail_train_df.to_csv("cnn_1.csv") # 檔名：cnn_1、cnn_2、cnn_3
new_cnn_dailymail_train_df.head()

,index,id,article,highlights
0,77926,dcea8becc6865676d300e82465e6832bdde3dcc7,A Libyan militant on Monday pleaded not guilty...,"Ahmed Abu Khattala, 43, pleaded not guilty to ..."
1,204254,946a91fd631c140f167c57861ba0a0f8247fa454,Chicago Bears star Chris Conte has said that h...,"Conte, who has suffered two concussions this s..."
2,30599,570343bebc7eec103f901ad9342747f3e028d179,"By . Lizzie Edmonds . PUBLISHED: . 09:42 EST, ...",Injured servicemen home from Walking With The ...
3,2045,05f6f7186f7a62875dc516ebdf0f99d5fc97e704,By . Daily Mail Reporter . PUBLISHED: . 13:01 ...,Kristyn Caddell says news stations are put off...
4,3610,0a6ecea9a1132a4cf7124cfb0c07efbe4450086c,Ralph Ellis runs you through what we've learne...,Jack Wilshere was named man of the match in En...


#### 讀取500篇文章

In [ ]:
new_cnn_dailymail_train_df = pd.read_csv('cnn_1.csv', delimiter=',')
new_cnn_dailymail_train_df.head()

In [ ]:
dailymail_articles = new_cnn_dailymail_train_df['article']
dailymail_summary = new_cnn_dailymail_train_df['highlights']
print(dailymail_articles)
print(dailymail_summary)

0         A Libyan militant on Monday pleaded not guilty...
1         Chicago Bears star Chris Conte has said that h...
2         By . Lizzie Edmonds . PUBLISHED: . 09:42 EST, ...
3         By . Daily Mail Reporter . PUBLISHED: . 13:01 ...
4         Ralph Ellis runs you through what we've learne...
                                ...                        
287108    Brendan Rodgers never took his eyes off Jordon...
287109    By . Darren Boyle . A senior Chinese tax offic...
287110    Los Angeles, California (CNN) -- Lawyers for M...
287111    By . Ashley Collman . PUBLISHED: . 09:09 EST, ...
287112    By . Matt Chorley, Mailonline Political Editor...
Name: article, Length: 287113, dtype: object
0         Ahmed Abu Khattala, 43, pleaded not guilty to ...
1         Conte, who has suffered two concussions this s...
2         Injured servicemen home from Walking With The ...
3         Kristyn Caddell says news stations are put off...
4         Jack Wilshere was named man of the match in E

### BillSum

In [ ]:
!unzip BillSum.zip

Archive:  BillSum.zip
  inflating: catest_processed.csv    
  inflating: ustest_processed.csv    
  inflating: ustrain_processed.csv   


In [ ]:
billSum_train_df = pd.read_csv('ustrain_processed.csv', delimiter=',')
# billSum_train_df.head()

In [ ]:
# 打亂順序，選擇前500筆資料
shuffle_billSum_train_df = shuffle(billSum_train_df)
new_billSum_train_df = shuffle_billSum_train_df[:500]
new_billSum_train_df.index.name="original_id"
# 轉CSV檔
new_billSum_train_df.to_csv("billSum_3.csv") # 檔名：billSum_1、billSum_2、billSum_3
new_billSum_train_df.head()

,bill_id,clean_text,summary,sum_len,text_len,cleantext_len
original_id,,,,,,
16387,113_s1250,SECTIONHEADER SHORT TITLE. This Act may be cit...,Transportation and Regional Infrastructure Pro...,765,17180,13290
14136,114_hr5637,SECTIONHEADER SHORT TITLE. This Act may be cit...,One Percent Spending Reduction Act of 2016 Thi...,1250,11561,8800
12780,110_hr2030,SECTIONHEADER SHORT TITLE. This Act may be cit...,Missouri African American Expulsion Commission...,781,8318,7200
14128,106_hr894,SECTIONHEADER SHORT TITLE This Act may be cite...,Makes the above inapplicable if the convicted ...,829,7158,5389
1078,109_hr5016,SECTIONHEADER SHORT TITLE. This Act may be cit...,Las Cienegas Enhancement Act - Requires the Se...,1932,7019,5892


#### 讀取500篇文章

In [ ]:
new_billSum_train_df = pd.read_csv('billSum_1.csv', delimiter=',')
new_billSum_train_df.head()

,original_id,bill_id,clean_text,summary,sum_len,text_len,cleantext_len
0,8742,111_s3765,SECTIONHEADER TABLE OF CONTENTS. The table of ...,"Title I: Benefit Matters - Extends, for those ...",2238,13295,10675
1,18053,112_hr3397,SECTIONHEADER SHORT TITLE; TABLE OF CONTENTS. ...,Cabin Fee Act of 2012 - Directs the Secretary ...,702,12253,8444
2,2025,108_hr3712,SECTIONHEADER SHORT TITLE. This Act may be cit...,United States Seaport Multiyear Security Enhan...,353,6833,5647
3,786,114_s1010,SECTIONHEADER SHORT TITLE. This Act may be cit...,"Small Business Mergers, Acquisitions, Sales, a...",1126,12539,6647
4,9296,113_s1611,SECTIONHEADER SHORT TITLE. This Act may be cit...,Federal Data Center Consolidation Act of 2013 ...,2778,17115,10437


In [ ]:
billsum_articles = new_billSum_train_df['clean_text']
billsum_summary = new_billSum_train_df['summary']
print(billsum_articles)
print(billsum_summary)

0      SECTIONHEADER TABLE OF CONTENTS. The table of ...
1      SECTIONHEADER SHORT TITLE; TABLE OF CONTENTS. ...
2      SECTIONHEADER SHORT TITLE. This Act may be cit...
3      SECTIONHEADER SHORT TITLE. This Act may be cit...
4      SECTIONHEADER SHORT TITLE. This Act may be cit...
                             ...                        
495    SECTIONHEADER SHORT TITLE. This Act may be cit...
496    SECTIONHEADER SHORT TITLE. This Act may be cit...
497    PROVIDING FOR BALANCED BUDGETS. Definitions. I...
498    SECTIONHEADER SHORT TITLE. This Act may be cit...
499    SECTIONHEADER SHORT TITLE. This Act may be cit...
Name: clean_text, Length: 500, dtype: object
0      Title I: Benefit Matters - Extends, for those ...
1      Cabin Fee Act of 2012 - Directs the Secretary ...
2      United States Seaport Multiyear Security Enhan...
3      Small Business Mergers, Acquisitions, Sales, a...
4      Federal Data Center Consolidation Act of 2013 ...
                             ...           

## 資料前處理

### tokenization

In [ ]:
# 移除標題等不必要的東西
def remove_TITLE( article ):
  article = re.sub( r'SECTIONHEADER SHORT TITLE.', '', article )
  article = re.sub( r'SECTIONHEADER', '', article )
  article = re.sub( r'TITLE I', '', article )
  article = re.sub( r'SHORT TITLE.', '', article )
  article = re.sub( r'SHORT TITLE;', '', article )
  article = re.sub( r'Short Title. ', '', article )
  article = re.sub( r'TABLE OF CONTENTS', '', article )
  article = re.sub( r'TABLE OF CONTENTS.', '', article )
  article = re.sub( r'\n', '', article )
  return article

CNN-DailyMail

In [ ]:
# 【cnn dailymail】
# 將所有文章分割成句子集合
articles_sentences = []
for row in dailymail_articles:
  row = remove_TITLE(row)
  tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
  articles_sentences.append( tokenizer.tokenize(row) )
# articles_sentences

BillSum

In [ ]:
# 【billsum】
# 先將所有文章分割成句子集合
articles_sentences = []
for row in billsum_articles:
  row = remove_TITLE(row)
  tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
  articles_sentences.append( tokenizer.tokenize(row) )
# articles_sentences

### Embedding

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

#### document embedding

CNN-DailyMail

In [ ]:
# 【cnn dailymail】 將所有文章進行embedding

doc_embedding = []
for article in dailymail_articles:
  doc_embedding.append( model.encode(article) )
np.shape(doc_embedding)

(500, 768)

BillSum

In [ ]:
# 【billsum_articles】將所有文章進行embedding

doc_embedding = []
for article in billsum_articles:
  doc_embedding.append( model.encode(article) )
np.shape(doc_embedding)

(500, 768)

#### sentence embedding

In [ ]:
sentence_embeddings = []
for row in articles_sentences:
  sentence_embeddings.append(model.encode(row))
print(np.shape(sentence_embeddings))
print(np.shape(sentence_embeddings[0]))

(500,)
(64, 768)


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:1970: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


* articles_sentences # 文章切成句子
* sentence_embeddings # 文章切成句子的嵌入

### 關鍵詞提取

#### 使用 MMR

In [ ]:
def mmr(doc_embedding, candidate_embeddings, article_keywords, top_n, diversity):
  # 與文檔的相關性
  keyw_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)
  # 關鍵詞的相關性
  keyw_similarity = cosine_similarity(candidate_embeddings)


  keywords_idx = [np.argmax(keyw_doc_similarity)]
  candidates_idx = [i for i in range(len(candidate_embeddings)) if i != keywords_idx[0]]
  for _ in range(top_n - 1):
    candidate_similarities = keyw_doc_similarity[candidates_idx, :]
    target_similarities = np.max(keyw_similarity[candidates_idx][:, keywords_idx], axis=1)

    ## MMR
    mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
    mmr_idx = candidates_idx[np.argmax(mmr)]

    keywords_idx.append(mmr_idx)
    candidates_idx.remove(mmr_idx)

  return [article_keywords[idx] for idx in keywords_idx]

CNN-DailyMail

In [ ]:
# 【cnn dailymail】
# 多篇文章測試
def keyword_extract(article):
  count = CountVectorizer( ngram_range= (1, 1) , stop_words="english" ).fit([article])
  candidates = count.get_feature_names_out()
  return candidates

articles_keywords = [] # 儲存每一篇的關鍵詞
top_n = 10
diversity = 0.2
for i, article in enumerate(dailymail_articles[:500]): 
  article_keywords = keyword_extract(article)
  candidate_embeddings = model.encode( article_keywords ) #將候選關鍵詞進行詞嵌入
  kword = mmr( [doc_embedding[i]], candidate_embeddings, article_keywords, top_n = top_n, diversity = diversity )
  articles_keywords.append(kword)  #儲存第 i 篇的前n個關鍵詞
print(np.shape(articles_keywords))

(500, 10)


BillSum

In [ ]:
# 【billsum】
# 多篇文章測試
def keyword_extract(article):
  count = CountVectorizer( ngram_range= (1, 1) , stop_words="english" ).fit([article])
  candidates = count.get_feature_names_out()
  return candidates

articles_keywords = [] # 儲存每一篇的關鍵詞
top_n = 10
diversity = 0.2
for i, article in enumerate(billsum_articles): # 先用前五篇試試
  article_keywords = keyword_extract(article)
  candidate_embeddings = model.encode( article_keywords ) #將候選關鍵詞進行詞嵌入
  kword = mmr( [doc_embedding[i]], candidate_embeddings, article_keywords, top_n = top_n, diversity = diversity )
  articles_keywords.append(kword)  #儲存第 i 篇的前n個關鍵詞
print(np.shape(articles_keywords))

(500, 10)


## 抽取式摘要

#### 分群

用 KMeansClusterer

In [ ]:
from nltk.cluster import KMeansClusterer

In [ ]:
# 使用餘弦距離進行 Kmeans
# 輸入文章的句子嵌入 & 分群的數量 ， 輸出每個句子的分群、質心向量
def sents_KMeansCluster(article_sents_embedding, num_clusters):
  
  kclusterer = KMeansClusterer(
          num_clusters, distance=nltk.cluster.util.cosine_distance,
          repeats=25,avoid_empty_clusters=True)
  assigned_clusters = kclusterer.cluster(article_sents_embedding, assign_clusters=True)
  centroid = kclusterer.means() # 相當於 kclusterer._centroid
  
  return assigned_clusters, centroid, kclusterer


In [ ]:
# 計算分群&質心
num_clusters = 4
articles_assigned_clusters = [] # 儲存每篇文章的分群結果(每一句是第幾群)

for article_id, sent_embeddings in enumerate(sentence_embeddings):
  if len(sent_embeddings)>=num_clusters:
    assigned_clusters, centroid, kclusterer = sents_KMeansCluster( sent_embeddings, num_clusters ) # 第i篇文章、分幾群
  else:
    assigned_clusters, centroid, kclusterer = sents_KMeansCluster( sent_embeddings, len(sent_embeddings) ) # 第i篇文章、分幾群
  articles_assigned_clusters.append([assigned_clusters, centroid, kclusterer])
# assigned_clusters # 每一句是第幾群

# 檢查用
print(articles_assigned_clusters[0][0])
print(np.shape( articles_assigned_clusters) )

[1, 1, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 2, 0, 3, 0, 2, 1, 1, 2, 1, 0, 2, 3, 0, 2, 3, 1, 3, 3, 3, 1, 3, 3, 1, 1, 3, 1, 1, 0, 2, 3, 3, 3, 3, 0, 2, 3, 0, 2, 1, 3, 1, 0, 2, 1, 1, 1, 3, 1, 1, 3, 1]
(500, 3)


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:1970: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


In [ ]:
# 顯示分群結果

articles_cluster_list = [] # 儲存每一篇文章的 assigned_clusters 、clustered_sentences 、clustered_sent_index 、clustered_distance 、distance_list

for article_id, article_cluster in enumerate(articles_assigned_clusters): # 第幾篇文章的分群結果
  size = len(articles_assigned_clusters[article_id][0])
  if size >= num_clusters:
    size = num_clusters
  clustered_sentences = [[] for i in range(size)] # 儲存被分到第i群的句子
  clustered_distance = [[] for i in range(size)] # 儲存被分到第i群的句子對質心的距離(按照群集分)
  clustered_sent_index = [[] for i in range(size)] # 儲存被分到第i群的句子的index
  distance_list = [] # 儲存句子向量和質心的距離(按照句子順序分)

  for sentence_id, cluster_id in enumerate( article_cluster[0] ): # 第 sentence_id 句被分到第 cluster_id 群
    clustered_sentences[cluster_id].append(articles_sentences[article_id][sentence_id]) # 儲存被分到第X篇文章第i群的句子
    distance = kclusterer._distance( sentence_embeddings[article_id][sentence_id], centroid[cluster_id] )
    distance_list.append( distance ) # 計算句子向量和質心的距離； sentence_embeddings[0]：文章0
    clustered_distance[cluster_id].append(distance) # 儲存被分到第i群的句子對質心的距離(按照群集分)
    clustered_sent_index[cluster_id].append( sentence_id )
    assigned_clusters = article_cluster[0]

  articles_cluster_list.append([assigned_clusters, clustered_sentences, clustered_sent_index, clustered_distance, distance_list])

# 檢查用
for i, cluster in enumerate(articles_cluster_list[0][1]):
  print("Cluster ", i+1)
  print(cluster)
  print("")

# 儲存第X篇文章的【assigned_clusters：每句分到哪群】、【clustered_sentences：每群的句子文字】、
#         【clustered_sent_index：每群句子原本的順序index】、【clustered_distance：每群句子向量和質心的距離(按照群集分)】、【distance_list：每個句子向量和質心的距離】
# articles_cluster_list

Cluster  1
['BENEFIT MATTERS Section 101.', 'Section 102.', 'Section 103.', 'Section 104.', 'I LOAN GUARANTY MATTERS Section 201.', 'Section 202.', 'II OTHER MATTERS Section 301.', 'BENEFIT MATTERS Section 101.', 'Section 102.', 'Section 103.', 'Section 104.', 'I LOAN GUARANTY MATTERS Section 201.', 'Section 202.', 'II OTHER MATTERS Section 301.']

Cluster  2
[' .', 'The table of contents for this Act is as follows:  Table of contents.', 'Separation or Release.', 'Paragraph (1)(A) of section 1968(a) of title 38, United States Code, is amended by striking clause (ii) and inserting the following new clause (ii): The date that is 2 years after the date of separation or release from such active duty or active duty for training, in the case of such a separation or release occurring on or after June 15, 2005.".', 'Paragraph (4) of such section is amended by striking subparagraph (B) and inserting the following new subparagraph (B): The date that is 2 years after the date of separation or rel

**參數整理**

* **assigned_clusters** # 每一句是第幾群
* **clustered_sentences** # 第i群的句子(文字)
* **clustered_sent_index** # 第i群的句子的index
* **clustered_distance** # 第i群的句子對質心的距離(按照群集分)
* **distance_list** # 每一句跟質心的距離

#### 新穎性分數

In [ ]:
# 新穎性分數
# 群集中最大的給1分

# 儲存每篇文章的新穎性分數
articles_novelty_score = []

for article_id, article_cluster in enumerate( articles_cluster_list ):

  score = copy.deepcopy( article_cluster[3] ) # clustered_distance
  for i, cluster_distance in enumerate( article_cluster[3] ):
    closest = cluster_distance.index(min(cluster_distance)) # 找到每一群中距離最近的句子index
    score[i][closest] = 1 # 給最近的1分

  # 將新穎性分數按照句子順序排
  size = np.size( sentence_embeddings[article_id],axis=0 )
  novelty_score = [0]*size
  for i, sent_index_list in enumerate( article_cluster[2] ): # clustered_sent_index
    for j, sent_index in enumerate(sent_index_list): # sent_index_list
      novelty_score[sent_index] = score[i][j]

  articles_novelty_score.append( novelty_score )

# articles_novelty_score

#### 位置分數

In [ ]:
# 文章(5篇)
pos_score = [] # 儲存每個文章中每一句的分數
for article_index in sentence_embeddings:
  max_index = np.size( article_index, axis=0 )-1
  score = [] # 儲存每一句的分數
  for sent_index, sent in enumerate(article_index):
    # print(sent_index)
    score.append( 1 - sent_index/max_index )
  pos_score.append(score)
print(np.shape(pos_score))

(500,)


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:1970: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


#### 重要群集

In [ ]:
articles_cluster_kword = [] # 儲存所有文章的每群關鍵詞數量
articles_ClusterWeight = [] # 儲存所有文章的每群權重

for article_id, article_cluster in enumerate( articles_cluster_list ):  


  # ## 將第i篇文章的關鍵詞進行embedding
  kword_embedding = model.encode( articles_keywords[article_id] )
  
  # 找出每一群中不重複的詞
  union_embedding_list = [] # 三維，儲存所在群集中，聯集單詞的embedding -> 如第一篇文章 有五群、三句、每句768維 (5, 3, 768)
  count_cluster_kword = []  # 計算關鍵詞數量
  ClusterWeight = []  # 群集權重
  threshold = 0.9 # 設定超過此數值即為關鍵詞

  ## 每群句子切 token 並進行聯集
  union_word_tokened = []
  for cluster in article_cluster[1]: # clustered_sentences
    union_tokened_list = []
    union_tokened = []
    for text in cluster:
      text_token = word_tokenize(text)
      union_tokened = union_tokened + text_token
      union_tokened_list = list( set(union_tokened) )
      union_tokened_list.sort(key=union_tokened.index)
  
    ## 將單詞embedding
    union_embedding = [] #一群集裡的單字嵌入(暫存用)
    for voc in union_tokened_list:
      union_embedding.append( model.encode(voc) ) # 將一個單詞進行embedding
    union_embedding_list.append( union_embedding )


    # 計算每個單詞向量與關鍵詞的相似度
    count = 0
    cluster_sim = cosine_similarity( union_embedding, kword_embedding )
    for voc in cluster_sim:
      if np.max(voc) > threshold:
        count+=1
    count_cluster_kword.append(count)
    ClusterWeight.append( math.log(1+math.log( 1 + count )) )

  articles_cluster_kword.append( count_cluster_kword )
  articles_ClusterWeight.append( ClusterWeight )

# print('每群的關鍵詞數量',articles_cluster_kword)
# print('每群的權重',articles_ClusterWeight)

In [ ]:
# 將重要群集分數分配給群集內的句子
articles_CW_score = [] # 儲存所有文章的群集權重分數
for article_id, sent_emd in enumerate(sentence_embeddings):
  size = np.size( sent_emd, axis=0) # 每一篇有幾句
  cluster_weight_score = [0]*size
  for i, sent_index_list in enumerate( articles_cluster_list[article_id][2] ): # clustered_sent_index
    for j, sent_index in enumerate(sent_index_list):
      cluster_weight_score[sent_index] = articles_ClusterWeight[article_id][i]
  articles_CW_score.append(cluster_weight_score)
# print(articles_CW_score)

#### 主題相關性分數

In [ ]:
threshold = 0.8 # 設定超過此數值即為關鍵詞
article_SW_list = [] # 紀錄每篇文章的每群的句子權重
topk_sort_CW_list = [] # 儲存每篇文章的前m個重要群集

for article_id, CW_score in enumerate( articles_ClusterWeight ): # articles_cluster_kword：每篇文章的每群權重 ； article_id：第X篇文章 ； CW_score：每群權重
  
  # ## 將第i篇文章的關鍵詞進行embedding
  kword_embedding = model.encode( articles_keywords[article_id] )

  # 排序群集，選擇前m個群集
  CW_sort = np.argsort( -np.asarray(CW_score) ) # CW_sort：存放權重大到小的群集，如：[3 1 2 4 0]，表示第3群權重最大
  # print('排序後前m個群集：', CW_sort)

  # Top m
  m = 3
  topk_sort_CW = copy.deepcopy(CW_sort[:m]) # topk_sort_CW：前m個群集
  topk_sort_CW.sort()
  topk_sort_CW_list.append(topk_sort_CW)


  # 將前m個群集中的每個句子單詞做嵌入
  
  SW_list = [] # 紀錄一篇文章裡每一群的句子權重
  
  ## 1. 先將前m個群集裡的 每個群集 中的每個句子單詞進行聯集(讓句子內的單詞不重複)
  for i, clust_index in enumerate(topk_sort_CW): # clust_index：第x個群集
    count_sent_kword = []  # 計算每一句的關鍵詞數量
    sent_weight = []  # 計算每一句的句子權重

    for i, clustered_sentences in enumerate(articles_cluster_list[article_id][1][clust_index]): # clustered_sentences # 第X個群集的句子
      text_token = word_tokenize(clustered_sentences) # 每群句子切 token
      union_text = list( set(text_token) ) # 將單詞進行聯集
      union_text.sort(key=text_token.index)

  ## 2. 將句子每個單詞進行嵌入
      sent = model.encode(union_text)
      
  ## 3. 計算每個句子的單詞向量與關鍵詞的相似度 
      count = 0
      sent_sim = cosine_similarity(sent,kword_embedding)

  ## 4. 計算句子裡有多少keyword (如果相似度大於X值，count++)
      for voc in sent_sim:
        if np.max(voc) > threshold:
          count+=1
      count_sent_kword.append(count)
      sent_weight.append( math.log(1+math.log( 1 + count )) )
    SW_list.append( sent_weight )
  article_SW_list.append( SW_list )
 
# print( article_SW_list )
# print( 'topk_sort_CW_list',topk_sort_CW_list )

In [ ]:
# 將重要句子分數分配給群集內的句子
articles_SW_score = [] # 儲存每篇文章的重要句子分數(依照句子順序)

for article_id, sent_ebd in enumerate(sentence_embeddings):
  size = np.size(sent_ebd , axis=0) # 每篇文章有多少句子
  sent_weight_score = [0]*size
  for i, clust_index in enumerate( topk_sort_CW_list[article_id] ):
    for j, sent_index in enumerate( articles_cluster_list[article_id][2][clust_index] ): # clustered_sent_index[clust_index]
      sent_weight_score[sent_index] = article_SW_list[article_id][i][j]
  articles_SW_score.append(sent_weight_score)
# print(articles_SW_score)

#### 將句子分數合併

**參數整理**

* **novelty_score** # 新穎性分數
* **pos_score** # 位置分數
* **cluster_weight_score** # 重要群集分數

In [ ]:
total_score = [] # 儲存所有句子的句子分數
for i in range(0,500):
  # pos_score ： 位置分數 ； articles_novelty_score：新穎性分數
  # articles_CW_score： 群集重要性 ； articles_SW_score：主題重要性
  total_score.append( 
      np.asarray( articles_novelty_score[i] ) + 
      np.asarray( articles_CW_score[i] ) + 
      np.asarray( pos_score[i] )  + 
      np.asarray( articles_SW_score[i] ) 
  )
total_score

#### 句子排序

In [ ]:
sent_sort_index = [] # 儲存所有文章的句子排序
for i, score in enumerate(total_score):
  # 一篇文章排序後的句子index
  sent_sort_index.append( np.argsort( -score ) )
sent_sort_index

#### Top K

In [ ]:
K = 5
articles_summary = [] # 儲存所有文章的summary
for i, score in enumerate(sent_sort_index):
  topk_sort = copy.deepcopy(score[:K])
  topk_sort.sort()
  summary = ""
  for j in topk_sort:
    summary += articles_sentences[i][j] + " "
  articles_summary.append(summary)

articles_summary[:5]

['A Libyan militant on Monday pleaded not guilty to charges arising from the 2012 Benghazi attacks that killed four Americans, including U.S. Ahmed Abu Khattala, 43, entered the plea through his lawyer in a 15-minute court proceeding before a federal judge. An 18-count grand jury indictment handed up last week makes Abu Khattala eligible for the death penalty if convicted. Suspect: This undated file image obtained from Facebook shows Ahmed Abu Khattala, an alleged leader of the deadly 2012 attacks on Americans in Benghazi, Libya . Abu Khattala is the first militant to be prosecuted for the Benghazi attacks. ',
 "Chicago Bears star Chris Conte has said that he believes playing in the NFL is worth any risk to his long-term health . Chicago Bears star Chris Conte, who has suffered two concussions this season, has said that he believes playing in the NFL is worth any risk to his long-term health. The 25-year-old defensive back told WBBM that he is happy to accept a shorter life span in ret

CNN-DailyMail

In [ ]:
standard_summary = dailymail_summary

BillSum

In [ ]:
standard_summary = billsum_summary

## 評估

In [ ]:
rouge = Rouge()

CNN-DailyMail

In [ ]:
# 【dailymail_articles】
Summaries = pd.concat([pd.DataFrame(standard_summary), pd.DataFrame(articles_summary)], axis=1).rename(columns= {'highlights':'standard_summary',0:'articles_summary'})

BillSum

In [ ]:
# 【billsum_articles】
Summaries = pd.concat([pd.DataFrame(standard_summary), pd.DataFrame(articles_summary)], axis=1).rename(columns= {'summary':'standard_summary',0:'articles_summary'})

In [ ]:
ourMethod = Summaries['articles_summary']
standardSummary = Summaries['standard_summary']

Summaries.head()

,standard_summary,articles_summary
0,"Ahmed Abu Khattala, 43, pleaded not guilty to ...",A Libyan militant on Monday pleaded not guilty...
1,"Conte, who has suffered two concussions this s...",Chicago Bears star Chris Conte has said that h...
2,Injured servicemen home from Walking With The ...,PUBLISHED: . This is the heartwarming moment a...
3,Kristyn Caddell says news stations are put off...,A reporter who had an affair with Dateline NBC...
4,Jack Wilshere was named man of the match in En...,Ralph Ellis runs you through what we've learne...


In [ ]:
# 匯出CSV檔
Summaries.to_csv("output_bill1_all.csv")

##### 單一分數測試

In [ ]:
# BillSum 500篇
# 新穎性
score = rouge.get_scores(ourMethod, standardSummary, avg=True)
PegsusRouge = pd.DataFrame(score).set_index([['recall','precision','f-measure']])*100
PegsusRouge

,rouge-1,rouge-2,rouge-l
recall,30.700486,13.405985,26.603036
precision,29.763386,13.669300,26.053556
f-measure,27.537748,11.820823,23.974941


In [ ]:
# BillSum 500篇
# 群集重要性
score = rouge.get_scores(ourMethod, standardSummary, avg=True)
PegsusRouge = pd.DataFrame(score).set_index([['recall','precision','f-measure']])*100
PegsusRouge

,rouge-1,rouge-2,rouge-l
recall,37.607801,18.934689,33.282763
precision,30.520035,14.860759,27.236951
f-measure,30.924354,14.779978,27.504798


In [ ]:
# BillSum 500篇
# 位置
score = rouge.get_scores(ourMethod, standardSummary, avg=True)
PegsusRouge = pd.DataFrame(score).set_index([['recall','precision','f-measure']])*100
PegsusRouge

,rouge-1,rouge-2,rouge-l
recall,25.413699,11.978793,22.331996
precision,30.131482,15.685316,26.907334
f-measure,24.678634,11.717800,21.849945


In [ ]:
# BillSum 500篇
# 主題重要性
score = rouge.get_scores(ourMethod, standardSummary, avg=True)
PegsusRouge = pd.DataFrame(score).set_index([['recall','precision','f-measure']])*100
PegsusRouge

,rouge-1,rouge-2,rouge-l
recall,47.800470,26.436144,42.563692
precision,27.562126,14.096413,24.711113
f-measure,32.845962,16.924776,29.385091


##### 多分數測試

###### 主題重要性 + 任一分數

In [ ]:
# BillSum 500篇
# 位置 + 主題重要性
score = rouge.get_scores(ourMethod, standardSummary, avg=True)
PegsusRouge = pd.DataFrame(score).set_index([['recall','precision','f-measure']])*100
PegsusRouge

,rouge-1,rouge-2,rouge-l
recall,42.547600,23.003249,38.045827
precision,29.073403,14.808758,26.178669
f-measure,31.991802,16.244989,28.723919


In [ ]:
# BillSum 500篇
# 群集重要性 + 主題重要性
score = rouge.get_scores(ourMethod, standardSummary, avg=True)
PegsusRouge = pd.DataFrame(score).set_index([['recall','precision','f-measure']])*100
PegsusRouge

,rouge-1,rouge-2,rouge-l
recall,47.086458,26.385899,42.146026
precision,27.863362,14.324095,25.075800
f-measure,32.784710,17.052183,29.452890


In [ ]:
# BillSum 500篇
# 新穎性 + 主題重要性
score = rouge.get_scores(ourMethod, standardSummary, avg=True)
PegsusRouge = pd.DataFrame(score).set_index([['recall','precision','f-measure']])*100
PegsusRouge

,rouge-1,rouge-2,rouge-l
recall,47.323637,26.254030,42.054494
precision,27.878592,14.242642,24.931976
f-measure,32.841138,16.908507,29.309006


###### 主題重要性 + 任兩分數

In [ ]:
# BillSum 500篇
# 新穎性 + 群集重要性 + 主題重要性
score = rouge.get_scores(ourMethod, standardSummary, avg=True)
PegsusRouge = pd.DataFrame(score).set_index([['recall','precision','f-measure']])*100
PegsusRouge

,rouge-1,rouge-2,rouge-l
recall,47.500347,26.441394,42.364735
precision,28.049043,14.384856,25.186553
f-measure,33.046767,17.089064,29.608461


In [ ]:
# BillSum 500篇
# 新穎性 + 位置 + 主題重要性
score = rouge.get_scores(ourMethod, standardSummary, avg=True)
PegsusRouge = pd.DataFrame(score).set_index([['recall','precision','f-measure']])*100
PegsusRouge

,rouge-1,rouge-2,rouge-l
recall,42.631172,23.017834,38.090385
precision,29.466181,15.123193,26.517375
f-measure,32.271900,16.488876,28.958150


In [ ]:
# BillSum 500篇
# 群集重要性 + 位置 + 主題重要性
score = rouge.get_scores(ourMethod, standardSummary, avg=True)
PegsusRouge = pd.DataFrame(score).set_index([['recall','precision','f-measure']])*100
PegsusRouge

,rouge-1,rouge-2,rouge-l
recall,43.495153,23.708100,38.987776
precision,28.809383,14.649444,25.946417
f-measure,32.126908,16.348717,28.884269


###### 主題重要性 + 全部

In [ ]:
# BillSum 500篇
# 新穎性 + 群集重要性 + 位置 + 主題重要性
score = rouge.get_scores(ourMethod, standardSummary, avg=True)
PegsusRouge = pd.DataFrame(score).set_index([['recall','precision','f-measure']])*100
PegsusRouge

,rouge-1,rouge-2,rouge-l
recall,43.319395,23.556093,38.769199
precision,28.996011,14.757000,26.064933
f-measure,32.277638,16.454823,28.958828
